In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Created on Mon Nov  22 10:07:38 2021
@author: earle
Title: Open3d Tutorial Point Cloud
Date: 22NOV2021
Author: Earle Lyons
Purpose: Examine Open3d point cloud functionality
    - Visualize point cloud
    - Voxel downsampling
    - Vertex normal estimation
    - Access estimated vertex normal
    - Crop point cloud
    - Paint point cloud
    - Point cloud distance
    - Bounding volumes
    - Convex hull
    - DBSCAN clustering
    - Plane segmentation
    - Hidden point removal
Inputs:
Outputs:
Notes:
###

In [1]:
import numpy as np
import glob
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
# List files in test_data directory
glob.glob('test_data/*.*')

['test_data\\bathtub_0154.ply',
 'test_data\\camera.json',
 'test_data\\camera_primesense.json',
 'test_data\\camera_trajectory.json',
 'test_data\\color.ply',
 'test_data\\depth.png',
 'test_data\\depth_syn.png',
 'test_data\\depth_syn_camera.json',
 'test_data\\download_file_list.json',
 'test_data\\download_test_data.cmake',
 'test_data\\download_test_data.py',
 'test_data\\download_utils.py',
 'test_data\\fragment.pcd',
 'test_data\\fragment.ply',
 'test_data\\image.PNG',
 'test_data\\knot.ply',
 'test_data\\lena_color.jpg',
 'test_data\\lena_gray.jpg',
 'test_data\\my_points.txt',
 'test_data\\renderoption.json',
 'test_data\\rs_default_config.json',
 'test_data\\simple.xyz',
 'test_data\\sphere.ply',
 'test_data\\tensors_compressed.npz',
 'test_data\\test_pose_graph.json',
 'test_data\\test_sample_ascii.ply',
 'test_data\\test_sample_custom.ply',
 'test_data\\test_sample_wrong_format.ply',
 'test_data\\test_tensorboard_plugin.zip']

# Point Cloud

# Visualize point cloud

In [7]:
# Read and visualize a point cloud
print("Load a ply point cloud, print it, and render it")
# Read frament.ply point cloud from a file
pcd = o3d.io.read_point_cloud("test_data/fragment.ply")
print(pcd)
print(np.asarray(pcd.points))
# Visualizes the point cloud
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

Load a ply point cloud, print it, and render it
PointCloud with 196133 points.
[[0.65234375 0.84686458 2.37890625]
 [0.65234375 0.83984375 2.38430572]
 [0.66737998 0.83984375 2.37890625]
 ...
 [2.00839925 2.39453125 1.88671875]
 [2.00390625 2.39488506 1.88671875]
 [2.00390625 2.39453125 1.88793314]]


## Voxel Downsampling

Voxel downsampling uses a regular voxel grid to create a uniformly downsampled point cloud from an input point cloud. It is often used as a pre-processing step for many point cloud processing tasks. The algorithm operates in two steps:

1. Points are bucketed into voxels.
2. Each occupied voxel generates exactly one point by averaging all points inside.

In [8]:
print("Downsample the point cloud with a voxel of 0.05")
# Downsample point cloud into voxels
downpcd = pcd.voxel_down_sample(voxel_size=0.05)
o3d.visualization.draw_geometries([downpcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

Downsample the point cloud with a voxel of 0.05


## Vertex Normal Estimation

In [9]:
print("Recompute the normal of the downsampled point cloud")
# Point normal estimation on downsampled point cloud
# Search radius is 0.1 (10cm) and maximun nearest neighbor is 30 neigbors
downpcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([downpcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024],
                                  point_show_normal=True)

Recompute the normal of the downsampled point cloud


## Access Estimated Vertex Normal

In [10]:
# Retrieve first (zero-based) estimated normal vectors from downcpcd variable
print("Print a normal vector of the 0th point")
print(downpcd.normals[0])

Print a normal vector of the 0th point
[ 0.85641574  0.01693013 -0.51600915]


In [11]:
# Retrieve first 10 estimated normal vectors from downcpcd variable
print("Print the normal vectors of the first 10 points")
print(np.asarray(downpcd.normals)[:10, :])

Print the normal vectors of the first 10 points
[[ 8.56415744e-01  1.69301342e-02 -5.16009150e-01]
 [-3.10071169e-01  3.92564590e-02 -9.49902522e-01]
 [-2.21066308e-01  2.07235365e-07 -9.75258780e-01]
 [-2.65577574e-01 -1.84601949e-01 -9.46250851e-01]
 [-7.91944115e-01 -2.92017206e-02 -6.09894891e-01]
 [-8.84912237e-02 -9.89400811e-01  1.15131831e-01]
 [ 6.28492508e-01 -6.12988948e-01 -4.78791935e-01]
 [ 7.28260110e-01 -4.73518839e-01 -4.95395924e-01]
 [-5.07368635e-03 -9.99572767e-01 -2.87844085e-02]
 [ 3.49295119e-01  1.16948013e-02 -9.36939780e-01]]


## Crop Point Cloud

In [13]:
print("Load a polygon volume and use it to crop the original point cloud")
# Reads the cropped.json file with the polygon selection area
vol = o3d.visualization.read_selection_polygon_volume(
    "test_data/Crop/cropped.json")
# Filters out points
chair = vol.crop_point_cloud(pcd)
o3d.visualization.draw_geometries([chair],
                                  zoom=0.7,
                                  front=[0.5439, -0.2333, -0.8060],
                                  lookat=[2.4615, 2.1331, 1.338],
                                  up=[-0.1781, -0.9708, 0.1608])

Load a polygon volume and use it to crop the original point cloud


In [24]:
# Display the cropped.json file
import json
f = open("test_data/Crop/cropped.json")
json.load(f)

{'axis_max': 4.022921085357666,
 'axis_min': -0.763413667678833,
 'bounding_polygon': [[2.6509309513852526, 0.0, 1.6834473132326844],
  [2.578642824691715, 0.0, 1.6892074266735244],
  [2.4625790337552154, 0.0, 1.6665777078297999],
  [2.2228544982251655, 0.0, 1.6168160446813649],
  [2.166993206001413, 0.0, 1.6115495157201662],
  [2.1167895865303286, 0.0, 1.6257706054969348],
  [2.0634657721747383, 0.0, 1.623021658624539],
  [2.0568612343437236, 0.0, 1.5853892911207643],
  [2.1605399001237027, 0.0, 0.9622899325508302],
  [2.1956669387205228, 0.0, 0.9557274604978507],
  [2.2191318790575583, 0.0, 0.8873444998210875],
  [2.248488184792592, 0.0, 0.8704280726701363],
  [2.6891234157295827, 0.0, 0.941406779889676],
  [2.7328692490470647, 0.0, 0.9877574067484025],
  [2.7129337547575547, 0.0, 1.0398850034649203],
  [2.7592174072415405, 0.0, 1.0692940558509485],
  [2.768921641945343, 0.0, 1.0953914441371593],
  [2.685145562545567, 0.0, 1.6307334122162018],
  [2.671477609998124, 0.0, 1.67552465708

## Paint Point Cloud

In [29]:
print("Paint chair")
# Paints points in a uniform color using RGB space
chair.paint_uniform_color([1, 0.706, 0])
o3d.visualization.draw_geometries([chair],
                                  zoom=0.7,
                                  front=[0.5439, -0.2333, -0.8060],
                                  lookat=[2.4615, 2.1331, 1.338],
                                  up=[-0.1781, -0.9708, 0.1608])

Paint chair


## Point Cloud Distance

In [31]:
# Load data
pcd = o3d.io.read_point_cloud("test_data/fragment.ply")
vol = o3d.visualization.read_selection_polygon_volume(
    "test_data/Crop/cropped.json")
chair = vol.crop_point_cloud(pcd)

# Computes the distance from each point in source point cloud (pcd) 
# to the closest point in the target point cloud (chair)
dists = pcd.compute_point_cloud_distance(chair)
# Distances are converted into a numpy array
dists = np.asarray(dists)
ind = np.where(dists > 0.01)[0]
pcd_without_chair = pcd.select_by_index(ind)
o3d.visualization.draw_geometries([pcd_without_chair],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

In [35]:
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

In [34]:
o3d.visualization.draw_geometries([chair],
                                  zoom=0.7,
                                  front=[0.5439, -0.2333, -0.8060],
                                  lookat=[2.4615, 2.1331, 1.338],
                                  up=[-0.1781, -0.9708, 0.1608])

In [39]:
dists

array([1.82659798, 1.83171046, 1.817216  , ..., 0.27083108, 0.27181889,
       0.27301031])

In [37]:
ind

array([     0,      1,      2, ..., 196130, 196131, 196132], dtype=int64)

## Bounding Volumes

In [59]:
# Get AxisAlignedBoundingBox from chain object
aabb = chair.get_axis_aligned_bounding_box()
aabb.color = (1, 0, 0)
# Get OrientedBoundingBox from chair object
obb = chair.get_oriented_bounding_box()
obb.color = (0, 1, 0)
o3d.visualization.draw_geometries([chair, aabb, obb],
                                  zoom=0.7,
                                  front=[0.5439, -0.2333, -0.8060],
                                  lookat=[2.4615, 2.1331, 1.338],
                                  up=[-0.1781, -0.9708, 0.1608])

### AxisAlignedBoundingBox

In [66]:
# Get AxisAlignedBoundingBox from chain object
aabb = chair.get_axis_aligned_bounding_box()
aabb.color = (1, 0, 0)
o3d.visualization.draw_geometries([chair, aabb],
                                  zoom=1.0,
                                  front=[0.5439, -0.2333, -0.8060],
                                  lookat=[2.4615, 2.1331, 1.338],
                                  up=[-0.1781, -0.9708, 0.1608])

### OrientedBoundingBox

In [62]:
# Get OrientedBoundingBox from chair object
obb = chair.get_oriented_bounding_box()
obb.color = (0, 1, 0)
o3d.visualization.draw_geometries([chair, obb],
                                  zoom=0.7,
                                  front=[0.5439, -0.2333, -0.8060],
                                  lookat=[2.4615, 2.1331, 1.338],
                                  up=[-0.1781, -0.9708, 0.1608])

## Convex Hull

In [81]:
# https://raw.githubusercontent.com/isl-org/Open3D/master/examples/python/open3d_tutorial.py
# http://graphics.stanford.edu/pub/3Dscanrep/bunny.tar.gz
bunny_mesh = o3d.io.read_triangle_mesh("test_data/bunny/reconstruction/bun_zipper.ply")
bunny_mesh.compute_vertex_normals()
pcl = bunny_mesh.sample_points_poisson_disk(number_of_points=2000)
# Using Qhull implementation, compute the convex hull of bunny_mesh point cloud
hull, _ = pcl.compute_convex_hull()
# A LineSet is created and visualized from a triangle mesh
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color((1, 0, 0))
o3d.visualization.draw_geometries([pcl, hull_ls])

## DBSCAN Clustering

In [83]:
import matplotlib.pyplot as plt

pcd = o3d.io.read_point_cloud("test_data/fragment.ply")

# DBSCAN clustering algorithm is implemented using cluster_dbscan and
# two parameters - eps (defines the distance to neighbors in a cluster)
# and min_points (deines the minimum number of points required to form
# a cluster)
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        pcd.cluster_dbscan(eps=0.02, min_points=10, print_progress=True))

# Assign the maximum number of labels (i.e. 9) to max_label)
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.455,
                                  front=[-0.4999, -0.1659, -0.8499],
                                  lookat=[2.1813, 2.0619, 2.0999],
                                  up=[0.1204, -0.9852, 0.1215])

generated new fontManager


[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 10
point cloud has 10 clusters
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [89]:
colors

array([[0.12156863, 0.46666667, 0.70588235, 1.        ],
       [0.12156863, 0.46666667, 0.70588235, 1.        ],
       [0.12156863, 0.46666667, 0.70588235, 1.        ],
       ...,
       [0.17254902, 0.62745098, 0.17254902, 1.        ],
       [0.17254902, 0.62745098, 0.17254902, 1.        ],
       [0.17254902, 0.62745098, 0.17254902, 1.        ]])

##  Plane Segmentation

In [90]:
pcd = o3d.io.read_point_cloud("test_data/fragment.pcd")
# Using segment_plane the plane is found with the largest support
# in the point cloud with three arguments - distance threshold
# (defines the maximum distance a point can have to an estimated
# plane to be considered an inlier), ransac_n (defines the number
# of points that are randomly sampled to estimate a plane), and
# num_interations (defines how often a random plane is sampled
# and verified).
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
                                  zoom=0.8,
                                  front=[-0.4999, -0.1659, -0.8499],
                                  lookat=[2.1813, 2.0619, 2.0999],
                                  up=[0.1204, -0.9852, 0.1215])

Plane equation: -0.06x + -0.10y + 0.99z + -1.06 = 0


## Hidden Point Removal

In [92]:
# https://raw.githubusercontent.com/isl-org/Open3D/master/examples/python/open3d_tutorial.py
# http://graphics.stanford.edu/pub/3Dscanrep/armadillo/Armadillo.ply.gz
armadillo_mesh = o3d.io.read_triangle_mesh("test_data/Armadillo.ply")
armadillo_mesh.compute_vertex_normals()

print("Convert mesh to a point cloud and estimate dimensions")
pcd = armadillo_mesh.sample_points_poisson_disk(5000)
diameter = np.linalg.norm(
    np.asarray(pcd.get_max_bound()) - np.asarray(pcd.get_min_bound()))
o3d.visualization.draw_geometries([pcd])

Convert mesh to a point cloud and estimate dimensions


In [93]:
print("Define parameters used for hidden_point_removal")
camera = [0, 0, diameter]
radius = diameter * 100

print("Get all points that are visible from given view point")
_, pt_map = pcd.hidden_point_removal(camera, radius)

print("Visualize result")
pcd = pcd.select_by_index(pt_map)
o3d.visualization.draw_geometries([pcd])

Define parameters used for hidden_point_removal
Get all points that are visible from given view point
Visualize result
